In [ ]:
!pip3 install "d2l==1.0.3"

# Neural Networks

## Multilayer Perceptrons

### Limits of Linear Models

<!-- ![OR vs XOR Graphic](../images/xor_problem.png) -->
<img src="../images/xor_problem.png" alt="OR vs XOR Graphic" width="500"/>

XOR involves two sets of 2D points: 
  - (0,0), (1,1) belong to one class.
  - (0,1), (1,0) belong to the other class.
  
Linear limitation
  - No single linear line can separate these classes.
  
Key takeaway
  - XOR is not linearly separable.
  - Nonlinear models (e.g., neural networks) are required.




### Multi-layer perceptron

<!-- ![An MLP with a hidden layer of five hidden units.](../d2l-en/img/mlp.svg) -->
<img src="../d2l-en/img/mlp.svg" alt="An MLP with a hidden layer of five hidden units." width="500"/>

**Notations**:
- Input: $\mathbf{X} \in \mathbb{R}^{1 \times d}$
- Hidden: $\mathbf{H} \in \mathbb{R}^{1 \times h}$
- Output: $\mathbf{O} \in \mathbb{R}^{1 \times q}$

**Parameters**:
- $\mathbf{W}^{(1)} \in \mathbb{R}^{d \times h}$, $\mathbf{b}^{(1)} \in \mathbb{R}^{1 \times h}$
- $\mathbf{W}^{(2)} \in \mathbb{R}^{h \times q}$, $\mathbf{b}^{(2)} \in \mathbb{R}^{1 \times q}$

**Forward Pass**:
$$
\mathbf{H} = \mathbf{X} \mathbf{W}^{(1)} + \mathbf{b}^{(1)}
$$
$$
\mathbf{O} = \mathbf{H} \mathbf{W}^{(2)} + \mathbf{b}^{(2)}
$$


**Affine Function Collapse**
- Equivalent single-layer model:
$$
\mathbf{O} = \mathbf{X} \mathbf{W} + \mathbf{b}
$$
with $\mathbf{W} = \mathbf{W}^{(1)} \mathbf{W}^{(2)}$ and $\mathbf{b} = \mathbf{b}^{(1)} \mathbf{W}^{(2)} + \mathbf{b}^{(2)}$.

**Nonlinear Activation**
- Introduce activation function $\sigma(x)$:
$$
\mathbf{H} = \sigma(\mathbf{X} \mathbf{W}^{(1)} + \mathbf{b}^{(1)})
$$
$$
\mathbf{O} = \mathbf{H} \mathbf{W}^{(2)} + \mathbf{b}^{(2)}
$$

**Multi-Layer Stacking**
$$
\mathbf{H}^{(1)} = \sigma_1(\mathbf{X} \mathbf{W}^{(1)} + \mathbf{b}^{(1)})
$$
$$
\mathbf{H}^{(2)} = \sigma_2(\mathbf{H}^{(1)} \mathbf{W}^{(2)} + \mathbf{b}^{(2)})
$$

### Universal Approximation Theorem

Let $C(X, \mathbb{R}^m)$ denote the set of continuous functions from a subset $X$ of Euclidean $\mathbb{R}^n$ to Euclidean space $\mathbb{R}^m$.  
Let $\sigma \in C(\mathbb{R}, \mathbb{R})$ be a continuous non-polynomial activation function. 

For every:
- $m, n \in \mathbb{N}$
- compact $X \subseteq \mathbb{R}^n$
- $f \in C(X, \mathbb{R}^m)$
- $\varepsilon > 0$

There exist:
- $k \in \mathbb{N}$
- $A \in \mathbb{R}^{k \times n}$
- $b \in \mathbb{R}^k$
- $C \in \mathbb{R}^{m \times k}$

Such that:
$$
\sup_{x \in X} \|f(x) - g(x)\| < \varepsilon
$$
Where:
$$
g(x) = C \cdot \left(\sigma \circ (A \cdot x + b)\right)
$$

### Activation functions

In [1]:
%matplotlib inline
import torch
from d2l import torch as d2l

ReLU provides a very simple nonlinear transformation:
$$\operatorname{ReLU}(x) = \max(x, 0).$$

In [ ]:
x = torch.arange(-8.0, 8.0, 0.1, requires_grad=True)
y = torch.relu(x)
d2l.plot(x.detach(), y.detach(), 'x', 'relu(x)', figsize=(5, 2.5))

In [ ]:
y.backward(torch.ones_like(x), retain_graph=True)
d2l.plot(x.detach(), x.grad, 'x', 'grad of relu', figsize=(5, 2.5))

The *sigmoid function* transforms those inputs
to outputs that lie on the interval (0, 1):

$$\operatorname{sigmoid}(x) = \frac{1}{1 + \exp(-x)}.$$

In [ ]:
y = torch.sigmoid(x)
d2l.plot(x.detach(), y.detach(), 'x', 'sigmoid(x)', figsize=(5, 2.5))

In [ ]:
x.grad.data.zero_()
y.backward(torch.ones_like(x),retain_graph=True)
d2l.plot(x.detach(), x.grad, 'x', 'grad of sigmoid', figsize=(5, 2.5))

The tanh (hyperbolic tangent)
function also squashes its inputs
between $-1$ and $1$.

$$\operatorname{tanh}(x) = \frac{1 - \exp(-2x)}{1 + \exp(-2x)}.$$

In [ ]:
y = torch.tanh(x)
d2l.plot(x.detach(), y.detach(), 'x', 'tanh(x)', figsize=(5, 2.5))

In [ ]:
x.grad.data.zero_()
y.backward(torch.ones_like(x),retain_graph=True)
d2l.plot(x.detach(), x.grad, 'x', 'grad of tanh', figsize=(5, 2.5))

## Implementation of a Multilayer Perceptron

In [8]:
import torch
from torch import nn
from d2l import torch as d2l

Implement an MLP
with one hidden layer and 256 hidden units

In [9]:
class MLPScratch(d2l.Classifier):
    def __init__(self, num_inputs, num_outputs, num_hiddens, lr, sigma=0.01):
        super().__init__()
        self.save_hyperparameters()
        self.W1 = nn.Parameter(torch.randn(num_inputs, num_hiddens) * sigma)
        self.b1 = nn.Parameter(torch.zeros(num_hiddens))
        self.W2 = nn.Parameter(torch.randn(num_hiddens, num_outputs) * sigma)
        self.b2 = nn.Parameter(torch.zeros(num_outputs))

Exercise: Implement the ReLU activation

Hints:
- Create a zero vector using `torch.zeros_like(X)`
- Use `torch.max()` with two arguments

In [ ]:
def relu(X):
    # TODO
    Y = torch.max(X, torch.zeros_like(X))
    # Y = None
    return Y

relu(torch.Tensor([-2., 0., 1.])) == torch.Tensor([0., 0., 1.])

Exercise: Implement our model

$$
\mathbf{H} = \sigma(\mathbf{X} \mathbf{W}^{(1)} + \mathbf{b}^{(1)})
$$
$$
\mathbf{O} = \mathbf{H} \mathbf{W}^{(2)} + \mathbf{b}^{(2)}
$$

Hints:
- Use ReLU for $\sigma$
- Use `torch.matmul` or the `@` operator

In [11]:
@d2l.add_to_class(MLPScratch)
def forward(self, X):
    X = X.reshape((-1, self.num_inputs))
    # TODO
    # H = None
    # Y = None
    H = relu(X @ self.W1 + self.b1)
    Y = H @ self.W2 + self.b2
    return Y

The training loop for MLPs

In [ ]:
model = MLPScratch(num_inputs=784, num_outputs=10, num_hiddens=256, lr=0.1)
data = d2l.FashionMNIST(batch_size=256)
trainer = d2l.Trainer(max_epochs=10)
trainer.fit(model, data)

Concise implementation using `nn.Sequential`

In [13]:
class MLP(d2l.Classifier):
    def __init__(self, num_outputs, num_hiddens, lr):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(nn.Flatten(), nn.LazyLinear(num_hiddens),
                                 nn.ReLU(), nn.LazyLinear(num_outputs))

The training loop

In [ ]:
model = MLP(num_outputs=10, num_hiddens=256, lr=0.1)
trainer.fit(model, data)

## Backpropagation
### Forward Propagation: Overview

**Input**  
  $$\mathbf{x} \in \mathbb{R}^d$$  


**Layers**

$$\mathbf{z}= \mathbf{W}^{(1)} \mathbf{x}$$
$$\mathbf{h}= \phi (\mathbf{z})$$
$$\mathbf{o}= \mathbf{W}^{(2)} \mathbf{h}$$

**Output**

$$\mathbf{o} = \mathbf{W}^{(2)} \phi (\mathbf{W}^{(1)} \mathbf{x})$$

### Loss and Regularization

**Loss Function**:  
  $$L = l(\mathbf{o}, y)$$  

**Regularization Term**:  
  $$s = \frac{\lambda}{2} \left(\|\mathbf{W}^{(1)}\|_\textrm{F}^2 + \|\mathbf{W}^{(2)}\|_\textrm{F}^2\right)$$  

**Objective Function**:  
  $$J = L + s$$  

### Computational Graph

<!-- ![Computational graph of forward propagation.](../d2l-en/img/forward.svg) -->
<img src="../d2l-en/img/forward.svg" alt="Computational graph of forward propagation." width="600"/>

### Backpropagation: Overview

**Weight update by gradient descent**

$$
w_{ij}^{(k)} \leftarrow w_{ij}^{(k)} - \eta \underbrace{\frac{\partial J}{\partial w_{ij}^{(k)}}}_{\text{gradient}}
$$

The step size $\eta$ is called *learning rate*.

**Gradient Calculation**

<!-- Objective Function Derivatives:  
$$\frac{\partial J}{\partial L} = 1 \quad \textrm{and} \quad \frac{\partial J}{\partial s} = 1$$ -->

Output Layer Gradient:  
$$\frac{\partial J}{\partial \mathbf{o}} = \frac{\partial J}{\partial L} \cdot \frac{\partial L}{\partial \mathbf{o}} = \frac{\partial L}{\partial \mathbf{o}}$$

Regularization Term Gradients:  
<!-- $$\frac{\partial s}{\partial \mathbf{W}^{(1)}} = \lambda \mathbf{W}^{(1)}, \quad \frac{\partial s}{\partial \mathbf{W}^{(2)}} = \lambda \mathbf{W}^{(2)}$$ -->
$$\frac{\partial s}{\partial w_{ij}^{(1)}} = \lambda w_{ij}^{(1)}, \quad \frac{\partial s}{\partial w_{ij}^{(2)}} = \lambda w_{ij}^{(2)}$$

**Backpropagation Through Layers**

<!-- Gradient w.r.t. $\mathbf{W}^{(2)}$:  
$$\frac{\partial J}{\partial \mathbf{W}^{(2)}} = \frac{\partial J}{\partial \mathbf{o}} \cdot \frac{\partial \mathbf{o}}{\partial \mathbf{W}^{(2)}} + \frac{\partial J}{\partial s} \cdot \frac{\partial s}{\partial \mathbf{W}^{(2)}} = \frac{\partial L}{\partial \mathbf{o}} \mathbf{h}^\top + \lambda \mathbf{W}^{(2)}$$

Hidden Layer Gradient:  
$$\frac{\partial J}{\partial \mathbf{h}} = \frac{\partial J}{\partial \mathbf{o}} \cdot \frac{\partial \mathbf{o}}{\partial \mathbf{h}} = {\mathbf{W}^{(2)}}^\top \frac{\partial J}{\partial \mathbf{o}}$$

Intermediate Variable Gradient:  
$$\frac{\partial J}{\partial \mathbf{z}} = \frac{\partial J}{\partial \mathbf{h}} \cdot \frac{\partial \mathbf{h}}{\partial \mathbf{z}} = \frac{\partial J}{\partial \mathbf{h}} \odot \phi'(\mathbf{z})$$

Gradient w.r.t. $\mathbf{W}^{(1)}$:  
$$\frac{\partial J}{\partial \mathbf{W}^{(1)}} = \frac{\partial J}{\partial \mathbf{z}} \cdot \frac{\partial \mathbf{z}}{\partial \mathbf{W}^{(1)}} + \frac{\partial J}{\partial s} \cdot \frac{\partial s}{\partial \mathbf{W}^{(1)}} = \frac{\partial J}{\partial \mathbf{z}} \mathbf{x}^\top + \lambda \mathbf{W}^{(1)}$$ -->

Gradient w.r.t. $w_{ij}^{(2)}$:  
$$\frac{\partial J}{\partial w_{ij}^{(2)}} = \frac{\partial J}{\partial \mathbf{o}} \cdot \frac{\partial \mathbf{o}}{\partial w_{ij}^{(2)}} + \frac{\partial J}{\partial s} \cdot \frac{\partial s}{\partial w_{ij}^{(2)}} = \frac{\partial L}{\partial o_i} h_j + \lambda w_{ij}^{(2)}$$

Hidden Layer Gradient:  
$$\frac{\partial J}{\partial \mathbf{h}} = \frac{\partial J}{\partial \mathbf{o}} \cdot \frac{\partial \mathbf{o}}{\partial \mathbf{h}} = \frac{\partial L}{\partial \mathbf{o}} \mathbf{W}^{(2)}$$

Intermediate Variable Gradient:  
$$\frac{\partial J}{\partial \mathbf{z}} = \frac{\partial J}{\partial \mathbf{h}} \cdot \frac{\partial \mathbf{h}}{\partial \mathbf{z}} = \frac{\partial J}{\partial \mathbf{h}} \operatorname{diag} \phi'(\mathbf{z})$$

Gradient w.r.t. $w_{ij}^{(1)}$:  
$$\frac{\partial J}{\partial w_{ij}^{(1)}} = \frac{\partial J}{\partial \mathbf{z}} \cdot \frac{\partial \mathbf{z}}{\partial w_{ij}^{(1)}} + \frac{\partial J}{\partial s} \cdot \frac{\partial s}{\partial w_{ij}^{(1)}} = \frac{\partial J}{\partial z_i} x_j + \lambda w_{ij}^{(1)}$$


## Training Stability and Initialization

- A deep network with $L$ layers: each layer $l$ is defined by a transformation $f_l$ with weights $\mathbf{W}^{(l)}$.
- Hidden layer output:
  $$\mathbf{h}^{(0)} = \mathbf{x}$$
  $$\mathbf{h}^{(l)} = f_l (\mathbf{h}^{(l-1)})$$
  $$\mathbf{o} = \mathbf{h}^{(L)}$$
- so output $\mathbf{o}$ becomes:
  $$\mathbf{o} = f_L \circ \cdots \circ f_1(\mathbf{x})$$

- Gradient of $\mathbf{o}$ w.r.t. $\mathbf{W}^{(l)}$:

  $$\partial_{\mathbf{W}^{(l)}} \mathbf{o} = \underbrace{\partial_{\mathbf{h}^{(L-1)}} \mathbf{h}^{(L)}}_{ \mathbf{M}^{(L)} \stackrel{\textrm{def}}{=}} \cdots \underbrace{\partial_{\mathbf{h}^{(l)}} \mathbf{h}^{(l+1)}}_{ \mathbf{M}^{(l+1)} \stackrel{\textrm{def}}{=}} \underbrace{\partial_{\mathbf{W}^{(l)}} \mathbf{h}^{(l)}}_{ \mathbf{v}^{(l)} \stackrel{\textrm{def}}{=}}.$$

- Numerical underflow/overflow due to product of matrices $\mathbf{M}^{(l)}$ with varying determinants.
  - **Exploding Gradient**: Large updates, model destruction.
  - **Vanishing Gradient**: Small updates, learning becomes impossible.

### Vanishing Gradients

- **Sigmoid activation**:  
  $$\sigma(x) = \frac{1}{1 + \exp(-x)}$$
  
- Sigmoid was popular due to its resemblance to biological neurons, but causes gradients to vanish.

In [ ]:
import torch
import d2l.torch as d2l

x = torch.arange(-8.0, 8.0, 0.1, requires_grad=True)
y = torch.sigmoid(x)
y.backward(torch.ones_like(x))

d2l.plot(x.detach().numpy(), [y.detach().numpy(), x.grad.numpy()],
         legend=['sigmoid', 'gradient'], figsize=(4.5, 2.5))

### Breaking the Symmetry

- **Symmetry** in MLPs causes weights to be updated identically, limiting expressiveness.
- **Example**: All hidden weights initialized as $\mathbf{W}^{(1)} = c$ leads to identical outputs and gradients.
- **Problem**: Gradient updates maintain symmetry, making hidden units redundant.
- **Solution**: Careful initialization or dropout breaks this symmetry.

<img src="../d2l-en/img/mlp.svg" alt="An MLP with a hidden layer of five hidden units." width="500"/>


### Xavier Initialization

- For layer output $o_i$:

  $$o_{i} = \sum_{j=1}^{n_\textrm{in}} w_{ij} x_j$$

- Variance of the output:

  $$
  \begin{aligned}
      \textrm{Var}[o_i] & = E[o_i^2] - (E[o_i])^2 \\
          & = \sum_{j=1}^{n_\textrm{in}} E[w^2_{ij} x^2_j] - 0 \\
          & = \sum_{j=1}^{n_\textrm{in}} E[w^2_{ij}] E[x^2_j] \\
          & = n_\textrm{in} \sigma^2 E[x^2_j].
  \end{aligned}
  $$

- To stabilize gradients: $n_\textrm{in} \sigma^2 = 1$

- Xavier Initialization:

  $$\frac{1}{2} (n_\textrm{in} + n_\textrm{out}) \sigma^2 = 1$$

**Xavier Initialization**

- Initialize weights by centered normal distribution with variance $\sigma$:

  $$\sigma = \sqrt{\frac{2}{n_\textrm{in} + n_\textrm{out}}}$$

- Or initialize by uniform distribution:

  $$U\left(-\sqrt{\frac{6}{n_\textrm{in} + n_\textrm{out}}}, \sqrt{\frac{6}{n_\textrm{in} + n_\textrm{out}}}\right)$$

Exercise: Apply Uniform Xavier initialization

Hints:
- Access linear layers using `self.net[i]` with an index `i`
- Access the weight of a linear layer using `linear.weight`
- Use `nn.init.xavier_uniform_()` with one argument


In [7]:
from torch import nn
from d2l import torch as d2l

class MLP(d2l.Classifier):
    def __init__(self, num_inputs, num_outputs, num_hiddens):
        super().__init__()
        self.save_hyperparameters()
        self.net = nn.Sequential(
            nn.Flatten(),
            nn.Linear(num_inputs, num_hiddens),
            nn.ReLU(),
            nn.Linear(num_hiddens, num_outputs),
        )
        # TODO
        nn.init.xavier_uniform_(self.net[1].weight)
        nn.init.xavier_uniform_(self.net[3].weight)


**Conclusion: training stability**

- Symmetry can lead to ineffective training.
- Xavier initialization ensures balanced variance in forward and backward pass
- ReLU activations help reduce vanishing gradients and speed convergence.